# Setup

Initial setup of required libraries

In [0]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
from keras import regularizers

import numpy as np
import pandas as pd
import os

# to make this notebook's output stable across runs
np.random.seed(42)

import time

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

from IPython.display import clear_output

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)

!pip install Pillow
from PIL import Image
import imageio

# GPU memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import GPUtil as GPU
clear_output()

### Mount the Working Directory

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Working_Directory = 'My Drive/Personal Projects/COVID-19 Project' #@param {type:"string"}
wd="/content/gdrive/"+Working_Directory
os.chdir(wd)

dirpath = os.getcwd()
print("current directory is : " + dirpath)

# Load the Images

Load Normal Patient Chest X-Ray Images

In [0]:
os.chdir(dirpath+"/data/normal")
normal_images_list = os.listdir()

normal_images = []
for image in normal_images_list:
    img = Image.open(image).convert('L')        ## convert to Gray Scale
    image_resize = img.resize((128, 128))
    data = np.asarray(image_resize, dtype="float64")
    normal_images.append(data)

Load COVID-19 Patient Chest X-Ray Images

In [0]:
os.chdir(dirpath+"/data/covid")
covid_images_list = os.listdir()

covid_images = []
for image in covid_images_list:
    img = Image.open(image).convert('L')        ## convert to Gray Scale
    image_resize = img.resize((128, 128))
    data = np.asarray(image_resize, dtype="float64")
    covid_images.append(data)

Let's view each image samples.

In [0]:
fig = plt.figure()
fig.suptitle('Normal Patient Chest X-ray')
plt.imshow(normal_images[0], cmap='gray') 

fig = plt.figure()
fig.suptitle('Covid Patient Chest X-ray')
plt.imshow(covid_images[0], cmap='gray') 

# Data Loading and Splitting

Define X and Y

In [0]:
normal = np.asarray(normal_images)
covid = np.asarray(covid_images)
X = np.vstack((normal, covid))
Y = np.vstack((np.zeros((normal.shape[0],1)),np.ones((covid.shape[0],1))))
class_names = ["Normal", "Covid"]

Split X and Y into Training Set, Validation Set and Testing Set

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=10)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.20, random_state=20)

Scale the data

In [0]:
X_train=X_train/255
X_val=X_val/255
X_test=X_test/255

# Model Implementation

Define a model

In [0]:
def build_model(n_hidden=1, n_neurons=30, reg=0.001, dropout=0.5, input_shape=[128, 128]):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu", kernel_regularizer=regularizers.l2(reg)))
        model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(1, activation="sigmoid"))
    optimizer = "sgd"
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics='accuracy')
    return model

keras_reg = keras.wrappers.scikit_learn.KerasClassifier(build_model)

Hypertune the model, using Random Search Method.

In [0]:
# Random Search
from sklearn.model_selection import RandomizedSearchCV

#Lets save the model and use TensorBoard
#Since the hyperparamter tuning process can take several hours, 
#saving the model at each epoch allows us to share the model with other
#and start traning again if something happens (e.g., we used the 12hr of GPU of Google Colab)
os.chdir(wd)
checkpoint_cb = keras.callbacks.ModelCheckpoint("model.h5", save_best_only=True)

#Lets read the pre-tranined model if there is any. 
try:  
    model = keras.models.load_model("model.h5") 
    print("Continue Training Model")
    # Do something with the file
except:
    print("Training Model from Scratch")

root_logdir = os.path.join(dirpath, "my_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

#What set of Hypeparamater you would like to test
param_distribs = {
    "n_hidden": [1,2,3,4,5],
    "n_neurons": [100,200,300,400],
    "reg":[0.0001, 0.001, 0.01],
    "dropout":[0.10, 0.25, 0.5]
}

tic = time.process_time()
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, cv=5, verbose=0)
history = rnd_search_cv.fit(X_train, Y_train, epochs=30, verbose=1, 
                            validation_data=(X_val, Y_val), 
                            callbacks=[checkpoint_cb, tensorboard_cb, keras.callbacks.EarlyStopping(patience=2)])
toc = time.process_time()
time_rnd=toc-tic

# Model Evaluation

Let's view our result using TensorBoard

In [0]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

Let's look at the best model found

In [0]:
rnd_search_cv.best_params_

In [0]:
#Load the best model. (the model is saved in model.h5)
model = keras.models.load_model("model.h5") 

Let's the model accuracy. 

In [0]:
train = model.evaluate(X_train, Y_train, verbose=0)
valid = model.evaluate(X_val, Y_val, verbose=0)
test = model.evaluate(X_test, Y_test, verbose=0)

print("Training Set Accuracy:   {0:.4%} & Loss:{1:.4f}".format(train[1], train[0]))
print("Validation Set Accuracy: {0:.4%} & Loss:{1:.4f}".format(valid[1], valid[0]))
print("Test Set Accuracy:       {0:.4%} & Loss:{1:.4f}".format(test[1], test[0]))